# Feed-Forward Neural Network

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from itertools import product
from pathlib import Path
from tqdm.auto import trange
import pickle

import numpy as np
import xarray as xr
import xarray.ufuncs as xf
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

from system_identification.ffnn import FeedForwardNeuralNetwork
from system_identification.load_assignment_data import load_net_example_ff

## 1D

In [3]:
def foo(x):
    return 0.8 * np.tanh(x * 0.4 - 0.7) + 2.1

# def f(x):
#     return x[0]**2

# def f(x):
#     return -0.8 * np.tanh(x[0] * 3) + x[1]**2 + 1

inputs = np.linspace(-5, 10, 10000)
reference_outputs = np.array(list(map(foo, inputs)))
reference_outputs_noisy = reference_outputs + np.random.normal(0, 0.1, size=reference_outputs.shape)

inputs = inputs.reshape(-1, 1, 1)
reference_outputs = reference_outputs.reshape(-1, 1, 1)
reference_outputs_noisy = reference_outputs_noisy.reshape(-1, 1, 1)

In [4]:
plt.figure()
plt.plot(inputs[:, 0, 0], reference_outputs_noisy[:, 0, 0], ".", markersize=1)
plt.plot(inputs[:, 0, 0], reference_outputs[:, 0, 0], ".", markersize=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Levenberg-Marquardt training

In [5]:
model = FeedForwardNeuralNetwork.new(
    n_inputs=1,
    n_outputs=1,
    n_hidden=3,
    input_range=[[-1, 1]],
)
model

<FeedForwardNeuralNetwork
 n_inputs=1
 n_hidden=3
 n_outputs=1>

In [6]:
model.train(
    inputs,
    reference_outputs_noisy,
    epochs=1000,
    goal=1e-6,
    train_log_freq=1,
    method="trainlm",
    mu=10.,
    alpha=0.95)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
model.training_log

<xarray.Dataset>
Dimensions:                (epoch: 1000, i: 10000, j: 1, k: 3, l: 2, m: 1, n: 4)
Coordinates:
  * epoch                  (epoch) int64 1 2 3 4 5 6 ... 996 997 998 999 1000
Dimensions without coordinates: i, j, k, l, m, n
Data variables: (12/13)
    gradient               (epoch) float64 3.987e+04 4.085e+04 ... 4.619e+04
    error_training_data    (epoch, i) float64 -2.075 -1.855 ... 0.02187 0.01679
    error_validation_data  (epoch, j) float64 nan nan nan nan ... nan nan nan
    error_training_jb      (epoch) float64 1.485e+03 1.299e+03 ... 0.3953 0.392
    error_training_jbp     (epoch) float64 0.0 1.035e-282 ... 0.8207 0.822
    error_training_skew    (epoch) float64 -0.7096 -0.6809 ... -0.0116 -0.01155
    ...                     ...
    error_validation_jb    (epoch) float64 nan nan nan nan ... nan nan nan nan
    error_validation_jbp   (epoch) float64 nan nan nan nan ... nan nan nan nan
    error_validation_skew  (epoch) float64 nan nan nan nan ... nan nan nan nan
    error_validation_kurt  (epoch) float64 nan nan nan nan ... nan nan nan nan
    weights_0              (epoch, k, l) float64 0.7157 -0.406 ... -9.295 -1.348
    weights_1              (epoch, m, n) float64 1.033 -0.7927 ... -1.07

In [8]:
fig = plt.figure()
outputs = model.evaluate(inputs)
plt.plot(inputs.squeeze(), reference_outputs.squeeze())
plt.plot(inputs.squeeze(), reference_outputs_noisy.squeeze(), ".", markersize=1)
plt.plot(inputs.squeeze(), outputs.squeeze())

fig, axs = plt.subplots(3, 1, figsize=(8, 6))
model.training_log.error_training_data.mean("i").plot(ax=axs[0])
axs[0].set_yscale("log")
model.training_log.error_training_jb.plot(ax=axs[1])
model.training_log.error_training_jbp.plot(ax=axs[2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Back propagation

In [9]:
model = FeedForwardNeuralNetwork.new(
    n_inputs=1,
    n_outputs=1,
    n_hidden=1,
    input_range=[[-1, 1]],
)
model

<FeedForwardNeuralNetwork
 n_inputs=1
 n_hidden=1
 n_outputs=1>

In [10]:
model.train(
    inputs,
    reference_outputs,
    epochs=10000,
    goal=1e-6,
    train_log_freq=1,
    method="trainbp",
    eta=0.001,
    alpha=1.001)

  0%|          | 0/10000 [00:00<?, ?it/s]

Goal met


In [12]:
fig = plt.figure()
outputs = model.evaluate(inputs)
plt.plot(inputs.squeeze(), reference_outputs.squeeze())
plt.plot(inputs.squeeze(), reference_outputs_noisy.squeeze(), ".", markersize=1)
plt.plot(inputs.squeeze(), outputs.squeeze())

fig, axs = plt.subplots(3, 1, figsize=(8, 6))
model.training_log.error_training_data.mean("i").plot(ax=axs[0])
axs[0].set_yscale("log")
model.training_log.error_training_jb.plot(ax=axs[1])
model.training_log.error_training_jbp.plot(ax=axs[2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2D

In [3]:
input_range = np.array([(-1, 1), (-3, 5)])


def foo(x):
    return -x[0]**2 + np.sin(x[1]*2)


resolution = 100
inputs = np.array(tuple(product(np.linspace(*input_range[0, :], resolution),
                                np.linspace(*input_range[1, :], resolution))))

reference_outputs = np.array(list(map(foo, inputs)))
reference_outputs_noisy = reference_outputs + np.random.normal(0, 0.1, size=reference_outputs.shape)

inputs = inputs[..., None]
reference_outputs = reference_outputs.reshape(-1, 1, 1)
reference_outputs_noisy = reference_outputs_noisy.reshape(-1, 1, 1)

In [4]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], reference_outputs.squeeze(), s=0.1)
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], reference_outputs_noisy.squeeze(), s=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Levenberg-Marquardt training

In [18]:
model = FeedForwardNeuralNetwork.new(
    n_inputs=2,
    n_outputs=1,
    n_hidden=10,
    input_range=input_range,
)
model

<FeedForwardNeuralNetwork
 n_inputs=2
 n_hidden=10
 n_outputs=1>

In [23]:
model.train(
    inputs,
    reference_outputs_noisy,
    epochs=500,
    goal=1e-6,
    train_log_freq=1,
    method="trainlm",
    mu=10.,
    alpha=0.99
)

  0%|          | 0/450 [00:00<?, ?it/s]

In [24]:
model.training_log

<xarray.Dataset>
Dimensions:                (epoch: 500, i: 10000, j: 1, k: 10, l: 3, m: 1, n: 11)
Coordinates:
  * epoch                  (epoch) int64 1 2 3 4 5 6 ... 495 496 497 498 499 500
Dimensions without coordinates: i, j, k, l, m, n
Data variables: (12/13)
    gradient               (epoch) float64 1.416e+05 1.393e+05 ... 2.303e+06
    error_training_data    (epoch, i) float64 0.07559 0.188 ... -0.001277
    error_validation_data  (epoch, j) float64 nan nan nan nan ... nan nan nan
    error_training_jb      (epoch) float64 261.6 1.154e+03 ... 0.9572 0.9566
    error_training_jbp     (epoch) float64 1.556e-57 2.465e-251 ... 0.6198
    error_training_skew    (epoch) float64 -0.341 0.7696 ... -0.0198 -0.01978
    ...                     ...
    error_validation_jb    (epoch) float64 nan nan nan nan ... nan nan nan nan
    error_validation_jbp   (epoch) float64 nan nan nan nan ... nan nan nan nan
    error_validation_skew  (epoch) float64 nan nan nan nan ... nan nan nan nan
    error_validation_kurt  (epoch) float64 nan nan nan nan ... nan nan nan nan
    weights_0              (epoch, k, l) float64 2.63 -0.04553 ... 0.6943
    weights_1              (epoch, m, n) float64 -0.3069 -8.351 ... 3.944 24.92

In [25]:
fig = plt.figure()
output = model.evaluate(inputs)
ax = fig.add_subplot(projection='3d')
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], reference_outputs.squeeze(), s=0.1)
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], output.squeeze(), s=0.1)

fig, axs = plt.subplots(3, 1, figsize=(8, 6))
model.training_log.error_training_data.mean("i").plot(ax=axs[0])
model.training_log.error_training_jb.plot(ax=axs[1])
model.training_log.error_training_jbp.plot(ax=axs[2])

axs[0].set_yscale("log")
axs[1].set_yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Back propagation training

In [13]:
model = FeedForwardNeuralNetwork.new(
    n_inputs=2,
    n_outputs=1,
    n_hidden=10,
    input_range=input_range
)
model

<FeedForwardNeuralNetwork
 n_inputs=2
 n_hidden=10
 n_outputs=1>

In [14]:
model.train(
    inputs,
    reference_outputs,
    epochs=10000,
    goal=1e-6,
    train_log_freq=1,
    method="trainbp",
    eta=.01,
    alpha=1.001
)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [17]:
fig = plt.figure()
output = model.evaluate(inputs)
ax = fig.add_subplot(projection='3d')
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], reference_outputs.squeeze(), s=0.1)
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], output.squeeze(), s=0.1)

fig, axs = plt.subplots(3, 1, figsize=(8, 6))
model.training_log.error_training_data.mean("i").plot(ax=axs[0])
model.training_log.error_training_jb.plot(ax=axs[1])
model.training_log.error_training_jbp.plot(ax=axs[2])

axs[0].set_yscale("log")
axs[1].set_yscale("log")
axs[2].set_yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …